# Explain Attacking BERT models using CAptum

Captum is a PyTorch library to explain neural networks
Here we show a minimal example using Captum to explain BERT models from TextAttack

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_5_Explain_BERT.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_5_Explain_BERT.ipynb)

In [1]:
import torch
from copy import deepcopy

In [2]:
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [3]:
#Optional: Install dependency CAptum
!pip install captum

In [4]:
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
from captum.attr import visualization as viz

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.set_device(device) 

cuda:0


In [6]:
dataset = HuggingFaceDataset("ag_news", None, "train")
original_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
original_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
model = HuggingFaceModelWrapper(original_model,original_tokenizer)

Using custom data configuration default
Reusing dataset ag_news (/p/qdata/jy2ma/.cache/textattack/datasets/ag_news/default/0.0.0/0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)
textattack: Loading datasets dataset ag_news, split train.


In [13]:
def get_text(tokenizer,input_ids,token_type_ids,attention_mask):
    list_of_text = []
    number = input_ids.size()[0]
    for i in range(number):
        ii = input_ids[i,].cpu().numpy()
        tt = token_type_ids[i,]
        am = attention_mask[i,]
        txt = tokenizer.decode(ii, skip_special_tokens=True)
        list_of_text.append(txt)
    return list_of_text
    
sel =2
batch_encoded = model.tokenizer([dataset[i][0]['text'] for i in range(sel)], padding=True, return_tensors="pt")
batch_encoded.to(device)
labels = [dataset[i][1] for i in range(sel)]

clone = deepcopy(model)
clone.model.to(device)

def calculate(input_ids,token_type_ids,attention_mask):
    #convert back to list of text
    return clone.model(input_ids,token_type_ids,attention_mask)[0]
    
# x = calculate(**batch_encoded)    

lig = LayerIntegratedGradients(calculate, clone.model.bert.embeddings)
# lig = InternalInfluence(calculate, clone.model.bert.embeddings)
# lig = LayerGradientXActivation(calculate, clone.model.bert.embeddings)

bsl = torch.zeros(batch_encoded['input_ids'].size()).type(torch.LongTensor).to(device)
labels = torch.tensor(labels).to(device)

attributions,delta = lig.attribute(inputs=batch_encoded['input_ids'],
                              baselines=bsl,
                              additional_forward_args=(batch_encoded['token_type_ids'], batch_encoded['attention_mask']),
                              n_steps = 10,
                              target = labels,
                              return_convergence_delta=True
                              )
atts = attributions.sum(dim=-1).squeeze(0)
atts = atts / torch.norm(atts)

{'input_ids': tensor([[  101,  2813,  2358,  1012,  6468, 15020,  2067,  2046,  1996,  2304,
          1006, 26665,  1007, 26665,  1011,  2460,  1011, 19041,  1010,  2813,
          2395,  1005,  1055,  1040, 11101,  2989,  1032,  2316,  1997, 11087,
          1011, 22330,  8713,  2015,  1010,  2024,  3773,  2665,  2153,  1012,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [  101, 18431,  2571,  3504,  2646,  3293, 13395,  1006, 26665,  1007,
         26665,  1011,  2797,  5211,  3813, 18431,  2571,  2177,  1010,  1032,
          2029,  2038,  1037,  5891,  2005,  2437,  2092,  1011, 22313,  1998,
          5681,  1032,  6801,  3248,  1999,  1996,  3639,  3068,  1010,  2038,
          5168,  2872,  1032,  2049, 29475,  2006,  2178,  2112,  1997,  1996,
          3006,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

In [14]:
atts = attributions.sum(dim=-1).squeeze(0)
atts = atts / torch.norm(atts)

In [15]:
from textattack.attack_recipes import PWWSRen2019
attack = PWWSRen2019.build(model)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [16]:
from textattack import Attacker

attacker = Attacker(attack, dataset)
attacker.attack_dataset()

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:04<00:44,  4.91s/it]

--------------------------------------------- Result 1 ---------------------------------------------
Business (96%) --> Sci/tech (68%)

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.

Wall St. suffer Claw Back Into the lightlessness (Reuters) Reuters - Short-sellers, Wall Street's dwindle\isthmus of ultra-cynics, are examine greenish again.




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:  20%|██        | 2/10 [00:10<00:40,  5.07s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
Business (100%) --> World (76%)

Fed minutes show dissent over inflation (USATODAY.com) USATODAY.com - Retail sales bounced back a bit in July, and new claims for jobless benefits fell last week, the government said Thursday, indicating the economy is improving from a midsummer slump.

eat hour testify protest over inflation (USATODAY.com) USATODAY.com - Retail sales bound hind a bit in July, and young claims for jobless welfare fell death workweek, the government said Thursday, indicating the economy is amend from a midsummer slump.




[Succeeded / Failed / Skipped / Total] 9 / 0 / 0 / 9:  90%|█████████ | 9/10 [00:49<00:05,  5.51s/it]

--------------------------------------------- Result 9 ---------------------------------------------
Business (100%) --> Sci/tech (100%)

Safety Net (Forbes.com) Forbes.com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a financial planner stopped by his desk to drop off brochures about insurance benefits available through his employer. But, at 32, "buying insurance was the furthest thing from my mind," says Riley.

Safety cyberspace (Forbes.com) Forbes.com - After earning a PH.D. in Sociology, Danny Bazil Riley started to work as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a financial planner stopped by his desk to drop off brochures about insurance benefits available through his employer. But, at 32, "buying insurance was the furthest thing from my mind," says Riley.




[Succeeded / Failed / Skipped / Total] 10 / 0 / 0 / 10: 100%|██████████| 10/10 [00:53<00:00,  5.32s/it]

--------------------------------------------- Result 10 ---------------------------------------------
Business (100%) --> World (71%)

Wall St. Bears Claw Back Into the Black  NEW YORK (Reuters) - Short-sellers, Wall Street's dwindling  band of ultra-cynics, are seeing green again.

Wall St. Bears chela Back Into the Black  novel YORK (Reuters) - Short-sellers, Wall Street's dwindling  band of ultra-cynics, are seeing green again.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 16.33% |
| Average num. words per input: | 38.5   |
| Avg num queries:              | 331.8  |
+-------------------------------+--------+
